In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from fuzzywuzzy import fuzz  # Make sure you have fuzzywuzzy installed: pip install fuzzywuzzy python-Levenshtein
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression # Example model, you can change this
from sklearn.metrics import classification_report

# ... (Load your dataframe 'df' here) ...

features = ['race_match', 'state_match', 'dob_similarity', 'sex_match', 'city_similarity', 'address_similarity', 'name_similarity']
X = df[features]
y = df['match']

# 1. Split the data FIRST
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)  # Adjust test_size as needed

# 2. Function to calculate fuzzy matching (now within the training set context)
def calculate_name_similarity(row):
    return fuzz.ratio(row['name_1'], row['name_2'])  # Assuming you have name_1 and name_2 columns

def calculate_address_similarity(row):
    return fuzz.ratio(row['address_1'], row['address_2']) # Assuming you have address_1 and address_2

# 3. Apply fuzzy matching ONLY to the TRAINING SET
X_train['name_similarity'] = X_train.apply(calculate_name_similarity, axis=1)
X_train['address_similarity'] = X_train.apply(calculate_address_similarity, axis=1)

# 4. Apply the SAME fuzzy matching logic to the TEST SET (using the training set's context)
X_test['name_similarity'] = X_test.apply(calculate_name_similarity, axis=1)
X_test['address_similarity'] = X_test.apply(calculate_address_similarity, axis=1)


# 5. One-Hot Encode Categorical Features (AFTER splitting)
categorical_features = ['race_match', 'sex_match', 'city_similarity', 'state_match'] # Add other categorical features as needed

# Use ColumnTransformer for more robust and cleaner one-hot encoding
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features) # Handle unknown values in test set
    ],
    remainder='passthrough'  # Keep the numerical features (dob_similarity, name_similarity, address_similarity)
)

X_train = preprocessor.fit_transform(X_train) # Fit and transform on TRAINING data
X_test = preprocessor.transform(X_test) # Transform TEST data using the fitted preprocessor


# 6. Train your model
model = LogisticRegression() # Or any other model
model.fit(X_train, y_train)

# 7. Evaluate the model
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))


# If you need to transform your data back to a pandas dataframe after one-hot encoding:
X_train_df = pd.DataFrame(X_train, columns=preprocessor.get_feature_names_out())
X_test_df = pd.DataFrame(X_test, columns=preprocessor.get_feature_names_out())

# ... (Rest of your code) ...